## Steer Function Comparison
In this notebook, we examine the planners using different steer functions.

In [16]:
%load_ext autoreload
%autoreload 2
from mpb import MPB, MultipleMPB
from plot_stats import plot_planner_stats, plot_smoother_stats
from utils import latexify
from table import latex_table
from definitions import *
import matplotlib as mpl
import sys, os
mpl.rcParams['mathtext.fontset'] = 'cm'
# make sure to not use Level-3 fonts
mpl.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
from copy import deepcopy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Moving AI Scenarios

In [21]:
def benchmark_moving_ai(scenario: str,
                        num_scenarios: int,
                        steer_function: str,
                        time_limit: float = 15):
    scenario_file = '/root/code/mpb/bin/moving_ai_scenarios/%s.map.scen' % scenario
    if not os.path.exists(scenario_file):
        raise Exception("Scenario file \"%s\" could not be found." % scenario_file)
    bench = "moving_ai_%s_%s" % (scenario, steer_function)
    total_filename = "%s/_total.json" % bench

    config = MPB.get_config()
    config["benchmark.moving_ai.active"] = True
    config["max_planning_time"] = time_limit
    config["benchmark.moving_ai.create_border"] = False
    # use point-based collision model for MovingAI scenarios
    config["env.collision.collision_model"] = 0

    pool = MultipleMPB()
    for i in range(num_scenarios):
        m = MPB()
        config["benchmark.moving_ai.start"] = i - num_scenarios
        config["benchmark.moving_ai.end"] = i - num_scenarios
        config["benchmark.moving_ai.scenario"] = scenario_file    
        m.update(deepcopy(config))
        m.set_steer_functions([steer_function])
        m.set_smoothers(['grips', 'ompl_bspline', 'ompl_shortcut', 'ompl_simplify_max'])
        #     m.set_planners(['theta_star'])
        #     m.set_planners(['sbpl_adstar'])
        pool.benchmarks.append(m)
    run_result = pool.run_parallel(bench, processes=20, limit_memory=True)

    # plot trajectories
    try:
        pool.visualize_trajectories(show_legend=True, fig_width=8, fig_height=8)
        plt.savefig("%s/%s_trajectories.pdf" % (bench, bench), bbox_inches='tight')
    except Exception as e:
        print("Error while visualizing trajectories:", e, file=sys.stderr)

    MPB.merge(pool.benchmarks, total_filename, make_separate_runs=True)

    # plot planner stats
    try:        
        plot_planner_stats(total_filename, save_file="%s/%s_stats.pdf" % (bench, bench))
    except Exception as e:
        print("Error while plotting planner statistics:", e, file=sys.stderr)

    # plot smoother stats
    try:
        plot_smoother_stats(total_filename,
                            save_file="%s/%s_smoother_stats.pdf" % (bench, bench),
                            separate_planners=False, num_colors=8,
                            combine_views=True, max_plots_per_line=3)
    except Exception as e:
        print("Error while plotting smoother statistics:", e, file=sys.stderr)

    # plot smoother stats (separated by planners)
    try:
        plot_smoother_stats(total_filename,
                            save_file="%s/%s_smoother_stats_separated.pdf" % (bench, bench),
                            separate_planners=True, num_colors=8, combine_views=True,
                            max_plots_per_line=1, fig_width=25, fig_height=6)
    except Exception as e:
        print("Error while plotting smoother statistics:", e, file=sys.stderr)
        
    # generate LaTeX table
    try:
        with open("%s/%s_table.txt" % (bench, bench), 'w') as tf:
            tf.write(latex_table(total_filename,
                                 row_label='\\textbf{Scenario: \\texttt{%s}} (%s steering, \\SI{%.1f}{\\minute} time limit)'

                                 % (latexify(scenario),
                                    latexify(steer_function_names[steer_function]),
                                    time_limit/60),
                                 time_limit=time_limit))
    except Exception as e:
        print("Error while generating table:", e, file=sys.stderr)

    return run_result

In [ ]:
scenario = 'Denver_1_256' #'Boston_1_1024'
num_scenarios = 50

time_limit_multiplier = 1

steer_function_times = [
    ('reeds_shepp', 15 * time_limit_multiplier),    
    ('dubins', 45 * time_limit_multiplier),    
    ('cc_reeds_shepp', 1.5 * 60 * time_limit_multiplier),    
    ('posq', 120 * time_limit_multiplier)
]

for steer_function, time_limit in steer_function_times:
    benchmark_moving_ai(scenario, num_scenarios, steer_function, time_limit)

Available memory: 256.73 GB, limiting each MPB process to 6.9% usage (17.83 GB).
Creating pool of 20 processes.
Running MPB with ID moving_ai_Denver_1_256_reeds_shepp_2 (log file at moving_ai_Denver_1_256_reeds_shepp/moving_ai_Denver_1_256_reeds_shepp_2.log)...
Running MPB with ID moving_ai_Denver_1_256_reeds_shepp_4 (log file at moving_ai_Denver_1_256_reeds_shepp/moving_ai_Denver_1_256_reeds_shepp_4.log)...
Running MPB with ID moving_ai_Denver_1_256_reeds_shepp_0 (log file at moving_ai_Denver_1_256_reeds_shepp/moving_ai_Denver_1_256_reeds_shepp_0.log)...
Running MPB with ID moving_ai_Denver_1_256_reeds_shepp_8 (log file at moving_ai_Denver_1_256_reeds_shepp/moving_ai_Denver_1_256_reeds_shepp_8.log)...
Running MPB with ID moving_ai_Denver_1_256_reeds_shepp_1 (log file at moving_ai_Denver_1_256_reeds_shepp/moving_ai_Denver_1_256_reeds_shepp_1.log)...
Running MPB with ID moving_ai_Denver_1_256_reeds_shepp_6 (log file at moving_ai_Denver_1_256_reeds_shepp/moving_ai_Denver_1_256_reeds_shep

Killed moving_ai_Denver_1_256_reeds_shepp_9 with planner sbpl_mha after 30.00s exceeded timeout.
Killed moving_ai_Denver_1_256_reeds_shepp_9 with planner sbpl_arastar after 30.01s exceeded timeout.
Killed moving_ai_Denver_1_256_reeds_shepp_1 with planner sbpl_arastar after 30.02s exceeded timeout.
Killed moving_ai_Denver_1_256_reeds_shepp_19 with planner sbpl_adstar after 30.02s exceeded timeout.


In [ ]:
pool.visualize_trajectories(show_legend=True)

In [ ]:
pool.benchmarks[19].visualize_trajectories(fig_width=20, fig_height=20)